In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm

In [2]:
day_i = pd.read_csv('data/Metro_testA/testA_record_2019-01-28.csv')

In [3]:
day_i.head()

,time,lineID,stationID,deviceID,status,userID,payType
0,2019-01-28 00:00:04,B,33,1587,0,Aad6ad59dfdd470bfdfdb0d2959db068b,0
1,2019-01-28 00:00:04,C,63,2933,0,Bab97d7b32891b8cb5451e80b73b23d70,1
2,2019-01-28 00:00:06,C,63,2934,0,Dda829d9f38d0a28f1e4ae84a34e846dc,3
3,2019-01-28 00:00:06,C,39,1839,0,Cc397069031ab762d75d613291cd68cda,2
4,2019-01-28 00:00:07,B,33,1588,0,Ba83c695d27b4d376bfde3f755e4b677f,1


In [4]:
predict = pd.read_csv('data/Metro_testA/testA_submit_2019-01-29.csv')

In [5]:
predict.head()

,stationID,startTime,endTime,inNums,outNums
0,0,2019-01-29 00:00:00,2019-01-29 00:10:00,NaN,NaN
1,0,2019-01-29 00:10:00,2019-01-29 00:20:00,NaN,NaN
2,0,2019-01-29 00:20:00,2019-01-29 00:30:00,NaN,NaN
3,0,2019-01-29 00:30:00,2019-01-29 00:40:00,NaN,NaN
4,0,2019-01-29 00:40:00,2019-01-29 00:50:00,NaN,NaN


In [6]:
19//10

1

In [7]:
day_i['time'] = pd.to_datetime(day_i.time)
day_i['minute_count'] = (day_i['time'].dt.minute+day_i['time'].dt.hour*60)//10

In [8]:
predict['time'] = pd.to_datetime(predict['startTime'])
predict['minute_count'] = (predict['time'].dt.minute+predict['time'].dt.hour*60)//10

In [9]:
predict.head()

,stationID,startTime,endTime,inNums,outNums,time,minute_count
0,0,2019-01-29 00:00:00,2019-01-29 00:10:00,NaN,NaN,2019-01-29 00:00:00,0
1,0,2019-01-29 00:10:00,2019-01-29 00:20:00,NaN,NaN,2019-01-29 00:10:00,1
2,0,2019-01-29 00:20:00,2019-01-29 00:30:00,NaN,NaN,2019-01-29 00:20:00,2
3,0,2019-01-29 00:30:00,2019-01-29 00:40:00,NaN,NaN,2019-01-29 00:30:00,3
4,0,2019-01-29 00:40:00,2019-01-29 00:50:00,NaN,NaN,2019-01-29 00:40:00,4


In [10]:
day_i['status'].value_counts()

0    1202647
1    1202247
Name: status, dtype: int64

In [11]:
out = day_i[day_i.status==0]
come = day_i[day_i.status==1]

In [12]:
come_count = come.groupby(['stationID','minute_count'],as_index=False).agg({'status':'count'})
out_count = out.groupby(['stationID','minute_count'],as_index=False).agg({'status':'count'})

In [13]:
come_count.head()

,stationID,minute_count,status
0,0,33,22
1,0,34,14
2,0,35,53
3,0,36,35
4,0,37,43


In [14]:
predict.head()

,stationID,startTime,endTime,inNums,outNums,time,minute_count
0,0,2019-01-29 00:00:00,2019-01-29 00:10:00,NaN,NaN,2019-01-29 00:00:00,0
1,0,2019-01-29 00:10:00,2019-01-29 00:20:00,NaN,NaN,2019-01-29 00:10:00,1
2,0,2019-01-29 00:20:00,2019-01-29 00:30:00,NaN,NaN,2019-01-29 00:20:00,2
3,0,2019-01-29 00:30:00,2019-01-29 00:40:00,NaN,NaN,2019-01-29 00:30:00,3
4,0,2019-01-29 00:40:00,2019-01-29 00:50:00,NaN,NaN,2019-01-29 00:40:00,4


In [15]:
predict_new = predict.merge(come_count,on=['stationID','minute_count'],how='left')
predict['inNums'] = list(predict_new['status'])
predict_new = predict.merge(out_count,on=['stationID','minute_count'],how='left')
predict['outNums'] = list(predict_new['status'])

In [16]:
predict.fillna(0,inplace=True)

In [17]:
predict[['stationID','startTime','endTime','inNums','outNums']].to_csv('result/testA_submit_2019-01-29.csv',index=False)

In [18]:
predict.dtypes

stationID                int64
startTime               object
endTime                 object
inNums                 float64
outNums                float64
time            datetime64[ns]
minute_count             int64
dtype: object

In [19]:
predict.shape

(11664, 7)